## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,San Andres,12.5847,-81.7006,81.81,76,43,19.84,CO,2021-12-01 02:01:25
1,1,Ostrovnoy,68.0531,39.5131,25.36,62,100,36.26,RU,2021-12-01 01:57:08
2,2,Isangel,-19.5500,169.2667,87.91,70,75,6.91,VU,2021-12-01 01:57:08
3,3,Nalut,30.3333,10.8500,48.70,56,4,8.72,LY,2021-12-01 01:57:08
4,4,Kidal,18.4411,1.4078,73.87,14,51,10.65,ML,2021-12-01 02:03:21


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("What is your minimum temperature preference?")
min_temp = float(input("min_temp: "))

print("What is your maximum temperature preference?")
max_temp = float(input("max_temp: "))


What is your minimum temperature preference?


min_temp:  50


What is your maximum temperature preference?


max_temp:  75


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Kidal,18.4411,1.4078,73.87,14,51,10.65,ML,2021-12-01 02:03:21
5,5,Mataura,-46.1927,168.8643,68.49,53,48,5.70,NZ,2021-12-01 02:03:01
6,6,Port Elizabeth,-33.9180,25.5701,63.10,87,75,2.80,ZA,2021-12-01 02:00:15
9,9,Busselton,-33.6500,115.3333,65.08,51,100,7.58,AU,2021-12-01 02:03:22
10,10,Lebu,-37.6167,-73.6500,52.81,92,8,10.76,CL,2021-12-01 02:03:23


In [6]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

223


Unnamed: 0    223
City          223
Lat           223
Lng           223
Max Temp      223
Humidity      223
Cloudiness    223
Wind Speed    223
Country       218
Date          223
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

218


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Kidal,18.4411,1.4078,73.87,14,51,10.65,ML,2021-12-01 02:03:21
5,5,Mataura,-46.1927,168.8643,68.49,53,48,5.70,NZ,2021-12-01 02:03:01
6,6,Port Elizabeth,-33.9180,25.5701,63.10,87,75,2.80,ZA,2021-12-01 02:00:15
9,9,Busselton,-33.6500,115.3333,65.08,51,100,7.58,AU,2021-12-01 02:03:22
10,10,Lebu,-37.6167,-73.6500,52.81,92,8,10.76,CL,2021-12-01 02:03:23


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Kidal,ML,73.87,18.4411,1.4078,
5,Mataura,NZ,68.49,-46.1927,168.8643,
6,Port Elizabeth,ZA,63.10,-33.9180,25.5701,
9,Busselton,AU,65.08,-33.6500,115.3333,
10,Lebu,CL,52.81,-37.6167,-73.6500,
14,Pimenteiras,BR,74.91,-6.2453,-41.4192,
15,Bambous Virieux,MU,73.22,-20.3428,57.7575,
22,Ribeira Grande,PT,62.73,38.5167,-28.7000,
24,Punta Arenas,CL,53.71,-53.1500,-70.9167,
25,Castro,BR,61.30,-24.7911,-50.0119,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Mataura,NZ,68.49,-46.1927,168.8643,Ellie's Villa
6,Port Elizabeth,ZA,63.10,-33.9180,25.5701,39 On Nile Guest House
9,Busselton,AU,65.08,-33.6500,115.3333,Observatory Guest House
10,Lebu,CL,52.81,-37.6167,-73.6500,Hostal Las Lilas
14,Pimenteiras,BR,74.91,-6.2453,-41.4192,Hotel rotativo pimentas


In [22]:
# 8a. Create the output File (CSV)
output_data_filev= "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("Vacation_Search.csv", index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))